In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from utils_function import load_Boston_housing_data
# load data
train_X, test_X, train_Y, test_Y = load_Boston_housing_data(feature_ind = [2,5])


Selected original features are ['INDUS' 'RM']


In [3]:
def np_cost(w1, w2, b, X, y):
    '''
    Evaluate the cost function in a non-vectorized manner for 
    inputs `X` and targets `y`, at weights `w1`, `w2` and `b`.
    '''
    costs = 0
    N = len(y)
    for i in range(len(y)):
        y_hat = w1*X[i][0] +w2*X[i][1] + b
        costs += (y_hat - y[i]) ** 2
    return costs / (2 * N)

In [4]:
np_cost(3, 5, 20, train_X, train_Y)


2256.1627893564355

In [5]:
def np_cost_vectorized(w1, w2, b, X, y):
    '''
    Evaluate the cost function in a vectorized manner for 
    inputs `X` and targets `t`, at weights `w1`, `w2` and `b`.
    '''
    N = len(y)
    w = np.asarray([w1, w2])
    y_hat = np.dot(X, w) + b
    return np.sum((y_hat - y)**2)/(2.0 * N)

In [6]:
np_cost_vectorized(3, 5, 20, train_X, train_Y)

2256.1627893564355

In [7]:
#tái tạo vector cho tập train
v1 = np.ones((train_X.shape[0], 1))
train_X_new = np.concatenate((train_X, v1), axis = 1)
print(train_X_new.shape)

(404, 3)


In [8]:
def np_solve_exactly(X, y):
    '''
    Solve linear regression exactly. (fully vectorized)
    
    Given `X` - NxD matrix of inputs
          `t` - target outputs
    Returns the optimal weights as a D-dimensional vector

    '''
    A = np.dot(X.T, X)
    c = np.dot(X.T, y)
    return np.matmul(np.linalg.inv(A), c)

In [9]:
np_solve_exactly(train_X_new, train_Y)

array([ -0.37785864,   7.83254633, -22.54414787])

In [23]:
def np_gradfn(weights, X, y):
    '''
    Given `weights` - a current "Guess" of what our weights should be
          `X` - matrix of shape (N,D) of input features
          `t` - target y values
    Return gradient of each weight evaluated at the current value
    '''
    N, D = X.shape
    y_hat = np.dot(X, weights)
    error = y_hat - y
    return np.matmul(np.transpose(X), error) / float(N)

In [24]:
def np_solve_via_gradient_descent(X, y, print_every=5000,
                               niter=100000, alpha=0.006):
    N, D = np.shape(X)
    print(X.shape)
    # initialize all the weights to zeros
    w = np.zeros([D])
    for k in range(niter):
        
        #TODO: Complete the below followed the above expressions
        dw = np_gradfn(w, X, y)
        w = w - alpha*dw
        
        if k % print_every == 0:
            print('Weight after %d iteration: %s' % (k, str(w)))
    return w

In [25]:
np_solve_via_gradient_descent(train_X_new, train_Y)

(404, 3)
Weight after 0 iteration: [1.30610257 0.88205579 0.13567129]
Weight after 5000 iteration: [-0.51684854  5.2408559  -4.51464123]
Weight after 10000 iteration: [-0.48594003  5.81719496 -8.52403558]
Weight after 15000 iteration: [ -0.46190493   6.26536798 -11.64182244]
Weight after 20000 iteration: [ -0.44321475   6.61387646 -14.06627713]
Weight after 25000 iteration: [ -0.42868089   6.88488377 -15.95158257]
Weight after 30000 iteration: [ -0.41737906   7.09562459 -17.41763456]
Weight after 35000 iteration: [ -0.40859052   7.25950095 -18.5576665 ]
Weight after 40000 iteration: [ -0.40175638   7.38693455 -19.44417861]
Weight after 45000 iteration: [ -0.39644201   7.48602951 -20.13354851]
Weight after 50000 iteration: [ -0.39230945   7.56308778 -20.66961661]
Weight after 55000 iteration: [ -0.38909589   7.62300987 -21.08647411]
Weight after 60000 iteration: [ -0.38659695   7.6696065  -21.41063101]
Weight after 65000 iteration: [ -0.38465373   7.70584099 -21.66270203]
Weight after 7

array([ -0.37902704,   7.81075969, -22.39258562])

In [26]:
#tensorflow
learning_rate = 0.005
training_epochs = 100000
display_step = 50

In [28]:
n_samples = train_X_new.shape[0]
#tf graph input
X = tf.placeholder(tf.float64, name = 'x1')
Y = tf.placeholder(tf.float64, name = 'y')
#set model weights
W = tf.Variable(np.zeros((train_X.shape[1], 1)), name = 'weights')

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
print(sess.run(W))
b = tf.Variable(np.zeros((1,1)), name = 'bias')

train_Y = train_Y.reshape(train_Y.shape[0], 1)

[[0.]
 [0.]]


In [29]:
#TODO: implement a gradient descent function
pred = tf.matmul(X, W) + b # y_hat

In [30]:
#TODO: implement a cost function

cost = tf.reduce_sum(tf.pow(pred - Y, 2))/(2*train_X.shape[0])


In [31]:
#TODO: implemement GD
learning_rate = 0.006
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [38]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    # Fit all training data
    for epoch in range(training_epochs):
        # TODO: implement sess.run for for full batch gradient descent
        sess.run(optimizer, feed_dict={X: train_X, Y: train_Y})
        if(epoch+1) % 50 == 0:
            c = sess.run(cost, feed_dict={X: train_X, Y: train_Y})
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(c), \
                "W=", sess.run(W), "b=", sess.run(b))
    print("Optimization Finished!")
    training_cost = sess.run(cost, feed_dict={X: train_X, Y: train_Y})
    print("Training cost=", training_cost, "W=", sess.run(W), "b=", sess.run(b), '\n')

Epoch: 0050 cost= 20.873978736 W= [[-0.50876485]
 [ 4.39846583]] b= [[0.56889287]]
Epoch: 0100 cost= 20.778875347 W= [[-0.55441209]
 [ 4.51326197]] b= [[0.5257399]]
Epoch: 0150 cost= 20.767386007 W= [[-0.555226  ]
 [ 4.52450369]] b= [[0.46819587]]
Epoch: 0200 cost= 20.756016501 W= [[-0.55481484]
 [ 4.53289293]] b= [[0.41039984]]
Epoch: 0250 cost= 20.744704091 W= [[-0.55437126]
 [ 4.54118386]] b= [[0.35273815]]
Epoch: 0300 cost= 20.733448443 W= [[-0.55392789]
 [ 4.54945184]] b= [[0.29522101]]
Epoch: 0350 cost= 20.722249272 W= [[-0.5534856]
 [ 4.557699 ]] b= [[0.23784835]]
Epoch: 0400 cost= 20.711106296 W= [[-0.55304442]
 [ 4.56592544]] b= [[0.1806198]]
Epoch: 0450 cost= 20.700019231 W= [[-0.55260436]
 [ 4.57413122]] b= [[0.12353502]]
Epoch: 0500 cost= 20.688987799 W= [[-0.55216539]
 [ 4.58231638]] b= [[0.06659363]]
Epoch: 0550 cost= 20.678011719 W= [[-0.55172753]
 [ 4.59048099]] b= [[0.00979528]]
Epoch: 0600 cost= 20.667090713 W= [[-0.55129077]
 [ 4.59862508]] b= [[-0.0468604]]
Epoch: 0

Epoch: 5300 cost= 19.851151616 W= [[-0.51477368]
 [ 5.27954507]] b= [[-4.78378858]]
Epoch: 5350 cost= 19.844379551 W= [[-0.51442975]
 [ 5.28595823]] b= [[-4.82840274]]
Epoch: 5400 cost= 19.837641465 W= [[-0.51408668]
 [ 5.29235527]] b= [[-4.87290482]]
Epoch: 5450 cost= 19.830937190 W= [[-0.51374448]
 [ 5.29873625]] b= [[-4.91729511]]
Epoch: 5500 cost= 19.824266554 W= [[-0.51340313]
 [ 5.3051012 ]] b= [[-4.9615739]]
Epoch: 5550 cost= 19.817629390 W= [[-0.51306264]
 [ 5.31145016]] b= [[-5.00574146]]
Epoch: 5600 cost= 19.811025529 W= [[-0.51272301]
 [ 5.31778318]] b= [[-5.04979806]]
Epoch: 5650 cost= 19.804454804 W= [[-0.51238423]
 [ 5.32410028]] b= [[-5.093744]]
Epoch: 5700 cost= 19.797917050 W= [[-0.5120463 ]
 [ 5.33040151]] b= [[-5.13757955]]
Epoch: 5750 cost= 19.791412099 W= [[-0.51170922]
 [ 5.33668692]] b= [[-5.18130498]]
Epoch: 5800 cost= 19.784939789 W= [[-0.51137298]
 [ 5.34295654]] b= [[-5.22492057]]
Epoch: 5850 cost= 19.778499955 W= [[-0.51103759]
 [ 5.34921041]] b= [[-5.268426

Epoch: 10300 cost= 19.317632465 W= [[-0.48432657]
 [ 5.84728045]] b= [[-8.73333008]]
Epoch: 10350 cost= 19.313537441 W= [[-0.48405912]
 [ 5.85226746]] b= [[-8.76802296]]
Epoch: 10400 cost= 19.309462965 W= [[-0.48379235]
 [ 5.85724193]] b= [[-8.80262869]]
Epoch: 10450 cost= 19.305408933 W= [[-0.48352624]
 [ 5.86220391]] b= [[-8.83714748]]
Epoch: 10500 cost= 19.301375243 W= [[-0.4832608 ]
 [ 5.86715343]] b= [[-8.87157957]]
Epoch: 10550 cost= 19.297361793 W= [[-0.48299603]
 [ 5.87209051]] b= [[-8.90592516]]
Epoch: 10600 cost= 19.293368481 W= [[-0.48273192]
 [ 5.87701519]] b= [[-8.94018448]]
Epoch: 10650 cost= 19.289395206 W= [[-0.48246848]
 [ 5.8819275 ]] b= [[-8.97435774]]
Epoch: 10700 cost= 19.285441869 W= [[-0.4822057 ]
 [ 5.88682747]] b= [[-9.00844515]]
Epoch: 10750 cost= 19.281508368 W= [[-0.48194358]
 [ 5.89171513]] b= [[-9.04244694]]
Epoch: 10800 cost= 19.277594604 W= [[-0.48168212]
 [ 5.89659051]] b= [[-9.07636331]]
Epoch: 10850 cost= 19.273700478 W= [[-0.48142131]
 [ 5.90145364]]

Epoch: 15250 cost= 18.997505627 W= [[-0.46085877]
 [ 6.28487531]] b= [[-11.77752831]]
Epoch: 15300 cost= 18.995016905 W= [[-0.46065028]
 [ 6.28876308]] b= [[-11.80457414]]
Epoch: 15350 cost= 18.992540670 W= [[-0.4604423 ]
 [ 6.29264107]] b= [[-11.83155203]]
Epoch: 15400 cost= 18.990076861 W= [[-0.46023485]
 [ 6.29650932]] b= [[-11.85846215]]
Epoch: 15450 cost= 18.987625414 W= [[-0.46002792]
 [ 6.30036786]] b= [[-11.88530467]]
Epoch: 15500 cost= 18.985186268 W= [[-0.45982151]
 [ 6.3042167 ]] b= [[-11.91207977]]
Epoch: 15550 cost= 18.982759361 W= [[-0.45961562]
 [ 6.30805588]] b= [[-11.9387876]]
Epoch: 15600 cost= 18.980344631 W= [[-0.45941025]
 [ 6.31188541]] b= [[-11.96542834]]
Epoch: 15650 cost= 18.977942018 W= [[-0.45920539]
 [ 6.31570532]] b= [[-11.99200217]]
Epoch: 15700 cost= 18.975551461 W= [[-0.45900104]
 [ 6.31951564]] b= [[-12.01850924]]
Epoch: 15750 cost= 18.973172898 W= [[-0.45879721]
 [ 6.32331638]] b= [[-12.04494972]]
Epoch: 15800 cost= 18.970806270 W= [[-0.4585939 ]
 [ 6.

Epoch: 20100 cost= 18.805998719 W= [[-0.44289009]
 [ 6.6199304 ]] b= [[-14.10839232]]
Epoch: 20150 cost= 18.804470922 W= [[-0.44272673]
 [ 6.6229765 ]] b= [[-14.129583]]
Epoch: 20200 cost= 18.802950791 W= [[-0.44256378]
 [ 6.62601495]] b= [[-14.15072045]]
Epoch: 20250 cost= 18.801438288 W= [[-0.44240124]
 [ 6.62904577]] b= [[-14.1718048]]
Epoch: 20300 cost= 18.799933374 W= [[-0.44223911]
 [ 6.63206897]] b= [[-14.19283619]]
Epoch: 20350 cost= 18.798436011 W= [[-0.44207738]
 [ 6.63508458]] b= [[-14.21381475]]
Epoch: 20400 cost= 18.796946162 W= [[-0.44191606]
 [ 6.63809261]] b= [[-14.23474061]]
Epoch: 20450 cost= 18.795463788 W= [[-0.44175515]
 [ 6.64109309]] b= [[-14.2556139]]
Epoch: 20500 cost= 18.793988852 W= [[-0.44159464]
 [ 6.64408603]] b= [[-14.27643476]]
Epoch: 20550 cost= 18.792521317 W= [[-0.44143454]
 [ 6.64707145]] b= [[-14.29720332]]
Epoch: 20600 cost= 18.791061146 W= [[-0.44127483]
 [ 6.65004937]] b= [[-14.3179197]]
Epoch: 20650 cost= 18.789608301 W= [[-0.44111553]
 [ 6.6530

...

Epoch: 79700 cost= 18.502275123 W= [[-0.38110259]
 [ 7.77205772]] b= [[-22.1233492]]
Epoch: 79750 cost= 18.502271321 W= [[-0.38109444]
 [ 7.77220967]] b= [[-22.12440625]]
Epoch: 79800 cost= 18.502267539 W= [[-0.38108631]
 [ 7.77236123]] b= [[-22.12546065]]
Epoch: 79850 cost= 18.502263775 W= [[-0.3810782 ]
 [ 7.77251242]] b= [[-22.12651239]]
Epoch: 79900 cost= 18.502260030 W= [[-0.38107012]
 [ 7.77266322]] b= [[-22.12756149]]
Epoch: 79950 cost= 18.502256305 W= [[-0.38106205]
 [ 7.77281365]] b= [[-22.12860796]]
Epoch: 80000 cost= 18.502252597 W= [[-0.381054 ]
 [ 7.7729637]] b= [[-22.1296518]]
Epoch: 80050 cost= 18.502248909 W= [[-0.38104598]
 [ 7.77311337]] b= [[-22.13069302]]
Epoch: 80100 cost= 18.502245239 W= [[-0.38103797]
 [ 7.77326267]] b= [[-22.13173162]]
Epoch: 80150 cost= 18.502241587 W= [[-0.38102998]
 [ 7.77341159]] b= [[-22.13276761]]
Epoch: 80200 cost= 18.502237954 W= [[-0.38102202]
 [ 7.77356014]] b= [[-22.133801]]
Epoch: 80250 cost= 18.502234339 W= [[-0.38101407]
 [ 7.77370

Epoch: 84800 cost= 18.501971038 W= [[-0.38036854]
 [ 7.78574517]] b= [[-22.21856814]]
Epoch: 84850 cost= 18.501968762 W= [[-0.38036224]
 [ 7.78586274]] b= [[-22.219386]]
Epoch: 84900 cost= 18.501966498 W= [[-0.38035595]
 [ 7.78598001]] b= [[-22.2202018]]
Epoch: 84950 cost= 18.501964245 W= [[-0.38034968]
 [ 7.78609698]] b= [[-22.22101556]]
Epoch: 85000 cost= 18.501962003 W= [[-0.38034342]
 [ 7.78621366]] b= [[-22.22182727]]
Epoch: 85050 cost= 18.501959772 W= [[-0.38033718]
 [ 7.78633005]] b= [[-22.22263694]]
Epoch: 85100 cost= 18.501957553 W= [[-0.38033095]
 [ 7.78644615]] b= [[-22.22344458]]
Epoch: 85150 cost= 18.501955345 W= [[-0.38032474]
 [ 7.78656195]] b= [[-22.22425019]]
Epoch: 85200 cost= 18.501953148 W= [[-0.38031855]
 [ 7.78667746]] b= [[-22.22505377]]
Epoch: 85250 cost= 18.501950962 W= [[-0.38031237]
 [ 7.78679269]] b= [[-22.22585534]]
Epoch: 85300 cost= 18.501948787 W= [[-0.3803062 ]
 [ 7.78690762]] b= [[-22.22665489]]
Epoch: 85350 cost= 18.501946623 W= [[-0.38030006]
 [ 7.78

Epoch: 89600 cost= 18.501797320 W= [[-0.37983013]
 [ 7.7957848 ]] b= [[-22.28841044]]
Epoch: 89650 cost= 18.501795916 W= [[-0.37982518]
 [ 7.79587715]] b= [[-22.28905285]]
Epoch: 89700 cost= 18.501794519 W= [[-0.37982024]
 [ 7.79596926]] b= [[-22.28969365]]
Epoch: 89750 cost= 18.501793129 W= [[-0.37981531]
 [ 7.79606114]] b= [[-22.29033284]]
Epoch: 89800 cost= 18.501791746 W= [[-0.37981039]
 [ 7.7961528 ]] b= [[-22.29097043]]
Epoch: 89850 cost= 18.501790370 W= [[-0.37980549]
 [ 7.79624422]] b= [[-22.29160641]]
Epoch: 89900 cost= 18.501789000 W= [[-0.3798006 ]
 [ 7.79633541]] b= [[-22.2922408]]
Epoch: 89950 cost= 18.501787638 W= [[-0.37979572]
 [ 7.79642637]] b= [[-22.29287359]]
Epoch: 90000 cost= 18.501786282 W= [[-0.37979086]
 [ 7.7965171 ]] b= [[-22.29350479]]
Epoch: 90050 cost= 18.501784934 W= [[-0.379786  ]
 [ 7.79660761]] b= [[-22.29413441]]
Epoch: 90100 cost= 18.501783592 W= [[-0.37978116]
 [ 7.79669789]] b= [[-22.29476245]]
Epoch: 90150 cost= 18.501782256 W= [[-0.37977633]
 [ 7.

Epoch: 94700 cost= 18.501685006 W= [[-0.37938402]
 [ 7.80410326]] b= [[-22.34627908]]
Epoch: 94750 cost= 18.501684165 W= [[-0.37938019]
 [ 7.80417471]] b= [[-22.34677613]]
Epoch: 94800 cost= 18.501683329 W= [[-0.37937636]
 [ 7.80424598]] b= [[-22.34727192]]
Epoch: 94850 cost= 18.501682497 W= [[-0.37937255]
 [ 7.80431707]] b= [[-22.34776648]]
Epoch: 94900 cost= 18.501681669 W= [[-0.37936875]
 [ 7.80438798]] b= [[-22.34825979]]
Epoch: 94950 cost= 18.501680845 W= [[-0.37936496]
 [ 7.80445871]] b= [[-22.34875186]]
Epoch: 95000 cost= 18.501680025 W= [[-0.37936117]
 [ 7.80452927]] b= [[-22.3492427]]
Epoch: 95050 cost= 18.501679210 W= [[-0.3793574 ]
 [ 7.80459965]] b= [[-22.3497323]]
Epoch: 95100 cost= 18.501678398 W= [[-0.37935363]
 [ 7.80466985]] b= [[-22.35022068]]
Epoch: 95150 cost= 18.501677591 W= [[-0.37934988]
 [ 7.80473988]] b= [[-22.35070782]]
Epoch: 95200 cost= 18.501676787 W= [[-0.37934613]
 [ 7.80480973]] b= [[-22.35119374]]
Epoch: 95250 cost= 18.501675988 W= [[-0.37934239]
 [ 7.8

Epoch: 99900 cost= 18.501616766 W= [[-0.37903293]
 [ 7.81064982]] b= [[-22.3918213]]
Epoch: 99950 cost= 18.501616268 W= [[-0.37902998]
 [ 7.81070483]] b= [[-22.39220395]]
Epoch: 100000 cost= 18.501615772 W= [[-0.37902704]
 [ 7.81075969]] b= [[-22.39258563]]
Optimization Finished!
Training cost= 18.50161577228284 W= [[-0.37902704]
 [ 7.81075969]] b= [[-22.39258563]] 

